In [10]:
# Use some required pathes
import pathlib
DATA_PATH = pathlib.Path().resolve() / "input_data_local" / "lips_idf_2023"
BENCH_CONFIG_PATH = pathlib.Path().resolve() / "configs" / "benchmarks" / "lips_idf_2023.ini"

In [8]:
import os
os.chdir("../Github/ml4physim_startingkit_powergrid")

In [11]:
BENCH_CONFIG_PATH.exists()

True

In [12]:
# Load the required benchmark datasets
from lips.benchmark.powergridBenchmark import PowerGridBenchmark

benchmark_kwargs = {"attr_x": ("prod_p", "prod_v", "load_p", "load_q"),
                    "attr_y": ("a_or", "a_ex", "p_or", "p_ex", "v_or", "v_ex"),
                    "attr_tau": ("line_status", "topo_vect"),
                    "attr_physics": None}

benchmark = PowerGridBenchmark(benchmark_name="Benchmark_competition",
                               benchmark_path=DATA_PATH,
                               load_data_set=True,
                               log_path=None,
                               config_path=BENCH_CONFIG_PATH,
                               **benchmark_kwargs
                              )

In [13]:
print(len(benchmark.train_dataset))

300000


In [22]:
# the next few lines are specific for each benchmark and each `AugmentedSimulator`
import grid2op
import warnings
from lightsim2grid.lightSimBackend import LightSimBackend
from lips.physical_simulator.dcApproximationAS import DCApproximationAS
with warnings.catch_warnings():
    warnings.filterwarnings("ignore")
    env = grid2op.make(benchmark.config.get_option("env_name"), test=True)
    grid_path = pathlib.Path(env.get_path_env()) / "grid.json"



dc_sim = DCApproximationAS(name="dc_approximation", 
                           benchmark_name="Benchmark_competition",
                           config_path=BENCH_CONFIG_PATH,
                           grid_path=grid_path,
                        #    backend=LightSimBackend,
                           ignore_assert=True
                           )

In [24]:
EVALUATION_PATH = os.path.join("input_data_local", "eval_results", "lips_idf_2023", "dc_approximation")
dc_metrics_per_dataset = benchmark.evaluate_simulator(augmented_simulator=dc_sim,
                                                       dataset="all", # other values : "val", "test", "test_ood_topo"
                                                       save_path=EVALUATION_PATH,
                                                       save_predictions=True
                                                      )

evaluate dc:   0%|▏                                                                        | 264/100000 [00:24<2:32:50, 10.88it/s]